In [87]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
%matplotlib tk
# % matplotlib inline
#plt.rcParams['animation.html'] = 'jshtml'
from IPython.display import HTML

from copy import deepcopy
import os
import sys
from functools import reduce
sys.path.append("/home/ful7rng/projects/transition/propagation")
from utils import pickle_load, get_npy, get_file
from utils.occ_map_utils import load_map, show_map, plot_occ_map, plot_trajectories, free_space
from utils.scene_utils import display_scenes_trajs, animate_scenes, get_scenes
from propagation.bofum import conditionalBOFUM, naiveBOFUM, BOFUMRealdata
from propagation.animation import TrackingAnimRealdata, Plot
from data_loader import get_map_crop

from utils import pickle_load, pickle_save
from utils.selecting_scene import SceneID, get_all_scenes_files


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [89]:
plt.ion()

In [211]:
path = '/local/data/scenes/100_11/classification.pkl'

In [212]:
cls = pickle_load(path)

In [213]:
for k, v in cls.items():
    print("{}: {}".format(k, len(v)))

Straight and turn: 268
Objects with diff. motion: 27
Through Doors: 6
Multiple Objects: 352
Straight: 816
Strange Behaviour: 6
Turn and Straight: 256
Missing Frames: 0
Turn: 366
processed: 195
Exclude: 1363
Noisy: 507


In [204]:
def containSceneID(scene_ids, scene_id):
    
    for id_ in scene_ids:
        if id_.file == scene_id.file and id_.idx == scene_id.idx:
            print("has scene_id {}/{}".format(scene_id.file, scene_id.idx))
            return True

In [205]:
scene_ids = np.array(cls['Straight and turn'] + cls['Straight'] + cls['Turn'] + cls['Turn and Straight'])
print("Found %d scene_ids for 4 motion classes" % len(scene_ids))
scene_ids_left = []
for idx, id_ in enumerate(scene_ids):
    if not containSceneID(cls['Objects with diff. motion'], id_):
        if not containSceneID(scene_ids_left, id_):
            scene_ids_left.append(id_)
        else:
            print("found duplicated scene_id {}/{}".format(id_.file, id_.idx))
print("Remove %d scene_ids since they have objects with different motion or they are duplicated" % (len(scene_ids)-len(scene_ids_left)))

Found 248 scene_ids for 4 motion classes
has scene_id /local/data/scenes/80_new/pioneer2/04_13/2017-04-13-10-33-31_5.npy/4
has scene_id /local/data/scenes/80_new/pioneer2/04_13/2017-04-13-13-33-31_14.npy/17
found duplicated scene_id /local/data/scenes/80_new/pioneer2/04_13/2017-04-13-13-33-31_14.npy/17
has scene_id /local/data/scenes/80_new/pioneer2/04_13/2017-04-13-10-33-31_5.npy/4
has scene_id /local/data/scenes/80_new/pioneer2/03_28/2017-03-28-13-19-24_12.npy/25
Remove 4 scene_ids since they have objects with different motion or they are duplicated


In [206]:
print("Got %d scenes with proper motions" % len(scene_ids_left))

Got 244 scenes with proper motions


In [207]:
def scene_id_to_scene(scene_id):
    scenes = np.load(scene_id.file)
    return scenes[scene_id.idx]

In [209]:
import random
scene = scene_id_to_scene(random.sample(scene_ids_left, 1)[0])
animate_scenes([scene])

### save scenes to file

In [210]:
scenes_list = []
for scene_id in scene_ids_left:
    scenes_list.append(scene_id_to_scene(scene_id))
scenes = np.array(scenes_list)
np.save("/home/ful7rng/projects/transition/data/244_scenes_for_test_from_80_new.npy", scenes)

### load scenes

In [137]:
sample_rate = 3
scenes = get_scenes(
                  random_file=True,
                  min_time_interval=3,
                  max_time_interval=1e6,
                  file_name='/home/ful7rng/projects/transition/data/scenes.npy',
                  sample_rate=sample_rate)[0]

load file: /home/ful7rng/projects/transition/data/scenes.npy
Found 1655 scenes in total
36
12000000
Found 1655 scenes (= 100.000%) whose interval is in range (3, 1000000)
Found 0 scenes (= 0.000%) whose interval does not match number of hits.


In [143]:
intervals = map(lambda scene: scene.end-scene.start, scenes)
_, _, _ = plt.hist(intervals)
plt.xlabel("s")

Text(0.5,0,u's')

In [141]:
idx = np.random.choice(np.arange(len(scenes)))
print(idx)
scene = scenes[idx]
map_ = scene.static_map
print(scene.hits.shape)
animate_scenes([scene])

1161
(15, 32, 32)


In [142]:
scene_copy = deepcopy(scene)
animate_scenes([BOFUMRealdata.scene_preprocessing(scene_copy)])